In [1]:
from smpc_secrets import ShamirSecretSharing, AdditiveSecretSharing, Vandermonde, P, RandPoly

In [2]:
import numpy as np
from numpy.linalg import inv
from scipy.interpolate import lagrange

In [3]:
class ShamirShare:
    def __init__(self,id, t, n, secrets):
        self.computations = []
        self.all_shares = []
        self.id = id
        for secret in secrets:
            shamir = ShamirSecretSharing(n=n, t=t)
            shares = shamir.generate(s=secret)
            self.all_shares.append(shares)
            self.computations.append((shamir, shares))
        self.shares = self.get_share(id)
        
    
    def get_share(self,id):
        return [shares[id][1] for shares in self.all_shares]
        
    def reconstruct(self):
        secrets = []
        for shamir, shares in self.computations:
            secrets.append(shamir.reconstruct(shares=shares))
        return secrets
    
    def _compute(self, other, op):
        assert len(self.shares) == len(other)
        for i in range(len(self.shares)):
            self.shares[i] = op(self.shares[i], other[i])
    
    def __add__(self,other):
        op = lambda x,y: x+y
        self._compute(self.shares, other, op)
        
    def __sub__(self,other):
        op = lambda x,y: x-y
        self._compute(self.shares, other, op)
        
    def __pow__(self, other):
        for i in range(len(self.shares)):
            self.shares[i] **= 2

In [4]:
class PeerToPeer:
    def __init__(self):
        pass
        
    def get_shares(self,X,Y):
        n=2
        t=2
        # Alice
        alice = ShamirShare(id=1, t=t, n=n, secrets=X)
        # Bob
        bob = ShamirShare(id=2, t=t, n=n, secrets=Y)

        # polynomials
        f = [alice.computations[i][0].poly for i in range(n)]
        g = [bob.computations[i][0].poly for i in range(n)]

        alice_id = 1
        alice_points = np.array([(f[i](alice_id), g[i](alice_id)) for i in range(n)])

        # Bob's polynomials
        bob_id = 2
        bob_points = np.array([(f[i](bob_id), g[i](bob_id)) for i in range(n)])

        # server computes d1(1)^2 and d2(1)^2
        server_id=3
        server_points = np.array([(f[i](server_id),g[i](server_id)) for i in range(n)])
        return np.stack((alice_points, bob_points, server_points))
    
    

In [5]:
for i in range(0,10,2):
    print((i, i+1))

(0, 1)
(2, 3)
(4, 5)
(6, 7)
(8, 9)


In [6]:
class SecureEuclideanDistance:
    def __init__(self, k, shares = []):
        self.k = k
        self.shares = np.array([])
        self.shares = np.append(self.shares, X)
        
    def add_shares(shares):
        self.shares = np.append(self.shares, shares)
        
    def compute(self,shares):
        d = [[np.power(point[0] - point[1],2) for point in share] for share in shares]
        dist = np.sum(d, axis=1)
        x = np.arange(1,len(dist)+1)
        y = dist
        f = lagrange(x, y)
        return f(0), np.sqrt(f(0))

In [7]:
class PeerToPeer:
    def __init__(self, k):
        self.k=k
        self.shares = np.array([])
        
       
    def get_shares(self,data):
        n=len(data)
        t=self.k
        for point in data:
            X = data[0]
            Y = data[1]
            shares = self._get_shares(X,Y)
            print(shares)
            self.shares = np.append(self.shares, shares)
        
        return shares
            
            
            
    def _get_shares(self,X,Y):
        n=2
        t=2
        # Alice
        alice = ShamirShare(id=1, t=t, n=n, secrets=X)
        # Bob
        bob = ShamirShare(id=2, t=t, n=n, secrets=Y)

        # polynomials
        f = [alice.computations[i][0].poly for i in range(n)]
        g = [bob.computations[i][0].poly for i in range(n)]

        alice_id = 1
        alice_points = np.array([(f[i](alice_id), g[i](alice_id)) for i in range(n)])

        # Bob's polynomials
        bob_id = 2
        bob_points = np.array([(f[i](bob_id), g[i](bob_id)) for i in range(n)])

        # server computes d1(1)^2 and d2(1)^2
        server_id=3
        server_points = np.array([(f[i](server_id),g[i](server_id)) for i in range(n)])
        return np.stack((alice_points, bob_points, server_points))

In [8]:
X=[1,3]
Y=[4,2]
data_point = np.array([X, Y])
data_point

array([[1, 3],
       [4, 2]])

In [9]:
p2p = PeerToPeer(k=2)
shares = p2p.get_shares(data_point)
dist = SecureEuclideanDistance(k=3)
shares, dist.compute(shares)

[[[ 177  213]
  [ 432  394]]

 [[ 353  422]
  [ 861  786]]

 [[ 529  631]
  [1290 1178]]]
[[[ 197 1260]
  [ 591  791]]

 [[ 393 2516]
  [1179 1580]]

 [[ 589 3772]
  [1767 2369]]]


(array([[[ 197, 1260],
         [ 591,  791]],
 
        [[ 393, 2516],
         [1179, 1580]],
 
        [[ 589, 3772],
         [1767, 2369]]]),
 (10.0, 3.1622776601683795))